In [2]:
# setup
# pip install --user ipyparallel
# ipcluster nbextension enable --user

# ipcluster start

In [3]:
import os
import ipyparallel as ipp

client = ipp.Client()
lb_view = client.load_balanced_view()

In [6]:
len(client)

16

In [38]:
@lb_view.parallel()
def f(x):
    """for parallel step source set identification on a group of 10 healthcodes"""
    import matplotlib
    import matplotlib.pyplot as plt
    import csv, os, pickle
    from os.path import isfile, join
    from os import listdir
    import numpy as np
    from datetime import datetime,timedelta
    from dateutil.parser import parse

    synapseCacheDir = "/scratch/PI/euan/projects/mhc/data/synapseCache_v2/"
    table_path = "/scratch/PI/euan/projects/mhc/data/tables/v2_data_subset/cardiovascular-HealthKitDataCollector-v1.tsv"

    def parse_healthkit_sources(file_path, HK_datatype):
        """Returns set of all (source, sourceIdentifier) that produces data of HK_datatype"""
        tally_set=set()
        #read in the data
        dtype_dict=dict()
        dtype_dict['names']=('startTime',
                             'endTime',
                             'type',
                             'value',
                             'unit',
                             'source',
                             'sourceIdentifier')
        dtype_dict['formats']=(datetime,
                               datetime,
                               'S36',
                               'i',
                               'S36',
                               'S36',
                               'S36')
        try:
            data=np.genfromtxt(file_path,
                               dtype=dtype_dict['formats'],
                               names=dtype_dict['names'],
                               delimiter=',',
                               loose=True,
                               invalid_raise=False, #needed since some lines are 1 column - issue with garmin...
                               converters={0:lambda x: parse(x),
                                           1:lambda x: parse(x)})
        except:
            print "There was a problem importing", file_path
            return tally_set
        #get the duration of each activity by day
        try:
            if data is not None and len(data):
                for row in range(len(data)):
                    if data['startTime'][row] is not None:
                        datatype = data['type'][row]
                        source = data['source'][row]
                        sourceIdentifier = data['sourceIdentifier'][row]
                        if datatype == HK_datatype:
                            tally_set.add((source, sourceIdentifier))
        except:
            print data, type(data)
        return tally_set

    def SubjectSources(data_table, subject):
        """Iterates over rows of data_table with subject's data and 
        finds the set of sourceIdentifiers that produce HKQuantityTypeIdentifierStepCounts"""
        sourses = set()
        data_subject = data_table[data_table["healthCode"]==subject]
        for row in data_subject:
            blob_name=row['data']
            if blob_name.endswith('NA'):
                continue 
            synapseCacheFile=get_synapse_cache_entry(synapseCacheDir,blob_name)
            sourses.update(parse_healthkit_sources(synapseCacheFile, "HKQuantityTypeIdentifierStepCount"))
        return sourses

    data_table= pickle.load( open(os.path.join("/home/hershman","cardiovascular-HealthKitDataCollector-v1.p"), "rb" ) )
    def ApplySubjects(subject_fn, data_table = data_table, function = SubjectSources):
        results={}

        subjects=open(subject_fn,'r').read().strip().split('\n')
        for subject in subjects:
            results[subject] = function(data_table, subject)
        return results

    subjects_forumula = "/scratch/PI/euan/projects/mhc/data/tables/v2_data_subset/subjects/healthkit_data/x%d"

    return ApplySubjects(data_table, subjects_forumula % x, SubjectSources)

In [39]:
ar = f.map(range(100))

In [41]:
ar.get_dict()

CompositeError: one or more exceptions from call to method: f
[0:apply]: UnicodeDecodeError: 'ascii' codec can't decode byte 0xa8 in position 144: ordinal not in range(128)
[10:apply]: UnicodeDecodeError: 'ascii' codec can't decode byte 0xa8 in position 144: ordinal not in range(128)
[1:apply]: UnicodeDecodeError: 'ascii' codec can't decode byte 0xa8 in position 144: ordinal not in range(128)
[7:apply]: UnicodeDecodeError: 'ascii' codec can't decode byte 0xa8 in position 144: ordinal not in range(128)
.... 96 more exceptions ...

In [49]:
ar.stdout

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [25]:
type(result)

ipyparallel.client.asyncresult.AsyncMapResult

In [32]:
ls /home/hershman

1 Exploring MHC Pipeline to find outliers.ipynb
2.1 Outlier Subject Data - diving in.ipynb
2.3 Outlier Subject Data - Sources.ipynb
2 Outlier Subject Data.ipynb
aggregators.py
aggregators.pyc
assemble_results_across_subjects.py
assemble_results_across_subjects.sh
cache.p
cardiovascular-HealthKitDataCollector-v1.p
filter_and_qc.py
filter_and_qc.sh
filters.py
get_activity_fraction_and_duration_appv2.activity_only.sh
get_activity_fraction_and_duration_appv2.healthkit.sh
get_activity_fraction_and_duration_appv2.py
get_objective_metrics_for_gwas.py
get_objective_metrics_for_gwas.sh
health_kit_metrics.py
load_summaries.py
measure_intervention_effects.R
motion_tracker_metrics.py
myheartcounts-master/
outprefix
Parallel.ipynb
qc_metrics.py
R/
R test (crashes).ipynb
sbatch.healthkit.sh
sbatch.motiontracker.sh
synapse_parser.py
synapse_parser.pyc
table_loader.py
table_loader.pyc
table_parser.py
table_parser.pyc


In [ ]:
from ipyparallel import Client

# The Client allows us to use the engines interactively.
# We simply pass Client the name of the cluster profile we
# are using.
In [2]: c = Client(profile='mycluster')
In [3]: v = c[:]

In [3]: c.ids
Out[3]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [4]: run pidigits.py

In [6]: files = [filestring % {'i':i} for i in range(1,16)]

In [7]: files
Out[7]:
['pi200m.ascii.01of20',
 'pi200m.ascii.02of20',
 'pi200m.ascii.03of20',
 'pi200m.ascii.04of20',
 'pi200m.ascii.05of20',
 'pi200m.ascii.06of20',
 'pi200m.ascii.07of20',
 'pi200m.ascii.08of20',
 'pi200m.ascii.09of20',
 'pi200m.ascii.10of20',
 'pi200m.ascii.11of20',
 'pi200m.ascii.12of20',
 'pi200m.ascii.13of20',
 'pi200m.ascii.14of20',
 'pi200m.ascii.15of20']

# download the data files if they don't already exist:
In [8]: v.map(fetch_pi_file, files)
